<a href="https://colab.research.google.com/github/AnastasiyaRahulina/Women-in-Data-Science-Datathon-March-2024/blob/main/WiDS_2024_smote_classifiers_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
pd.set_option('display.max_columns', None)

In [ ]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 7.9 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/WinDS/training_modified.csv', index_col=0)


In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/WinDS/test_modified.csv', index_col=0)

In [ ]:
train_data.drop(columns=['N02', 'farmer', 'never_married'], inplace = True)


In [ ]:
test_data.drop(columns=['N02', 'farmer', 'never_married'], inplace = True)

In [ ]:
# train_data = pd.read_csv('/content/drive/MyDrive/WinDS/csv_sets/data_simple.csv', index_col=0)
# test_data = pd.read_csv('/content/drive/MyDrive/WinDS/csv_sets/data_test_simple.csv', index_col=0)

In [ ]:
# train_data = pd.read_csv('/content/drive/MyDrive/WinDS/csv_sets/data_aug.csv', index_col=0)
# test_data = pd.read_csv('/content/drive/MyDrive/WinDS/csv_sets/data_test_aug.csv', index_col=0)

In [ ]:
train_data.head(5)

,DiagPeriodL90D,Division_East North Central,Division_East South Central,Division_Middle Atlantic,Division_Mountain,Division_New England,Division_Pacific,Division_South Atlantic,Division_West North Central,Division_West South Central,N02,Ozone,PM25,Region_Midwest,Region_Northeast,Region_South,Region_West,age_10_to_19,age_20s,age_30s,age_40s,age_50s,age_60s,age_70s,age_median,age_over_80,age_under_10,breast_cancer_diagnosis_desc_processed_malig neoplasm quadrant left femal breast,breast_cancer_diagnosis_desc_processed_malig neoplasm quadrant right male breast,breast_cancer_diagnosis_desc_processed_malig neoplasm quadrant unsp femal breast,breast_cancer_diagnosis_desc_processed_malig neoplm quadrant right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm axillari tail femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm axillari tail left femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm axillari tail right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm axillari tail unsp femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm breast,breast_cancer_diagnosis_desc_processed_malign neoplasm breast femal unspecifi,breast_cancer_diagnosis_desc_processed_malign neoplasm breast unspecifi site,breast_cancer_diagnosis_desc_processed_malign neoplasm breast unspecifi site femal,breast_cancer_diagnosis_desc_processed_malign neoplasm central portion breast femal,breast_cancer_diagnosis_desc_processed_malign neoplasm central portion femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm central portion left femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm central portion right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm central portion unsp femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm nippl areola femal,breast_cancer_diagnosis_desc_processed_malign neoplasm nippl areola left femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm nippl areola right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm nippl areola unsp femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm overlap site breast femal,breast_cancer_diagnosis_desc_processed_malign neoplasm ovrlp site left femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm ovrlp site right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm ovrlp site unsp femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm quadrant breast femal,breast_cancer_diagnosis_desc_processed_malign neoplasm quadrant femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm specifi site femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm unsp site right femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm unsp site unspecifi femal breast,breast_cancer_diagnosis_desc_processed_malign neoplasm unsp site unspecifi male breast,breast_cancer_diagnosis_desc_processed_malign neoplasm unspecifi site left femal breast,breast_cancer_diagnosis_desc_processed_secondari malign neoplasm breast,commute_time,density,disabled,divorced,education_bachelors,education_college_or_above,education_graduate,education_highschool,education_less_highschool,education_some_college,education_stem_degree,family_dual_income,family_size,farmer,female,health_uninsured,hispanic,home_ownership,home_value,hosp_rating,housing_units,income_household_100_to_150,income_household_10_to_15,income_household_150_over,income_household_15_to_20,income_household_20_to_25,income_household_25_to_35,income_household_35_to_50,income_household_50_to_75,income_household_5_to_10,income_household_75_to_100,income_household_median,income_household_six_figure,income_household_under_5,income_individual_median,labor_force_participation,limited_english,male,married,metastatic_cancer_category_Other or Unspecified Metastasis,metastatic_cancer_diagnosis_code_C770,metastatic_cancer_diagnosis_code_C771,

In [ ]:
 pd.set_option('display.max_rows', None)

In [ ]:
train_data.isnull().sum()

In [ ]:
train_data.shape, test_data.shape

((12906, 220), (5792, 219))

In [ ]:
train_data = pd.get_dummies(train_data, columns=['patient_race'])
test_data = pd.get_dummies(test_data, columns=['patient_race'])
train_data.shape, test_data.shape

((12906, 225), (5792, 224))

In [ ]:
# test_data = pd.get_dummies(test_data, columns=['patient_race'])


In [ ]:
# test_data.dtypes

In [ ]:
train_data.isnull().sum()

DiagPeriodL90D                                                                               0
Division_East North Central                                                                  0
Division_East South Central                                                                  0
Division_Middle Atlantic                                                                     0
Division_Mountain                                                                            0
Division_New England                                                                         0
Division_Pacific                                                                             0
Division_South Atlantic                                                                      0
Division_West North Central                                                                  0
Division_West South Central                                                                  0
Ozone                                             

In [ ]:
# train_data['Hospital Ownership'].fillna('Unknown', inplace=True)
# test_data['Hospital Ownership'].fillna('Unknown', inplace=True)


In [ ]:
# types_dict = train_data.dtypes.to_dict()
# test_dtypes = test_data.dtypes.to_dict()

# for key in types_dict:
#     if types_dict[key]=='O':
#         train_data[key] = train_data[key].astype('category')

# for key in test_dtypes:
#     if test_dtypes[key]=='O':
#         test_data[key] = test_data[key].astype('category')

In [ ]:
# train_data['Hospital Ownership'] = train_data['Hospital Ownership'].astype('category')
# test_data['Hospital Ownership'] = test_data['Hospital Ownership'].astype('category')

In [ ]:
X = train_data.drop(['patient_id','DiagPeriodL90D'], axis=1)
y = train_data['DiagPeriodL90D']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
X_train.shape, X_test.shape

((10324, 223), (2582, 223))

#SMOTE

In [ ]:
# cats = train_data.select_dtypes(exclude=np.number).columns.tolist()
# cats

[]

In [ ]:
# cats_idx = [X.columns.get_loc(col) for col in cats]
# cats_idx

[0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 78, 79, 80]

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE


ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py)

In [ ]:
from imblearn.over_sampling import SMOTE
# from imblearn.over_sampling import SMOTENC

# smote_nc = SMOTENC(categorical_features=cats_idx, random_state=10)
# X_new,y_new = smote_nc.fit_resample(X, y)

smote = SMOTE(random_state=10)
X_new,y_new = smote.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_new, y_new, test_size=0.2, random_state=10)
X_train.shape, X_test.shape

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/_param_validation.py)

In [ ]:
se = StandardScaler()
scaled_X_train = se.fit_transform(X_train)
scaled_X_test = se.transform(X_test)

#Random Forest Classifier

In [ ]:
parameters_rf = {'max_depth':10, 'min_samples_split':50, 'n_estimators':187}


In [ ]:
rf = RandomForestClassifier(**parameters_rf)

rf.fit(X_train, y_train)
y_pred = rf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
auc

0.8605255683786143

##tuning

In [ ]:
# def objective_rf(trial):
#     params = {'n_estimators': trial.suggest_int('n_estimators', low=100, high=500, step=100),
#               'max_features': trial.suggest_categorical('max_features', choices=['auto', 'sqrt']),
#               'max_depth': trial.suggest_int('max_depth', low=10, high=110, step=20),
#               'min_samples_split': trial.suggest_int('min_samples_split', low=2, high=10, step=2),
#               'min_samples_leaf': trial.suggest_int('min_samples_leaf', low=1, high=4, step=1)}

#     model = RandomForestClassifier(random_state=10, **params)

#     model.fit(X_train, y_train)
#     y_pred = model.predict_proba(X_test)[:, 1]
#     auc = roc_auc_score(y_test, y_pred)

#     return auc


def objective(trial):
    # Number of trees in random forest
    n_estimators = trial.suggest_int(name="n_estimators", low=100, high=500, step=100)

    # Maximum number of levels in tree
    max_depth = trial.suggest_int(name="max_depth", low=10, high=110, step=20)

    # Minimum number of samples required to split a node
    min_samples_split = trial.suggest_int(name="min_samples_split", low=2, high=10, step=2)

    # Minimum number of samples required at each leaf node
    min_samples_leaf = trial.suggest_int(name="min_samples_leaf", low=1, high=4, step=1)

    params = {
        "n_estimators": n_estimators,
        "max_depth": max_depth,
        "min_samples_split": min_samples_split,
        "min_samples_leaf": min_samples_leaf
    }
    model = RandomForestClassifier(random_state=10, **params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)

    return auc

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-02-28 17:01:22,821] A new study created in memory with name: no-name-deb56130-ad66-4fa5-9804-6c891699fada
[I 2024-02-28 17:01:54,309] Trial 0 finished with value: 0.8620439770878263 and parameters: {'n_estimators': 500, 'max_depth': 90, 'min_samples_split': 4, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8620439770878263.
[I 2024-02-28 17:02:06,777] Trial 1 finished with value: 0.8618888842707405 and parameters: {'n_estimators': 200, 'max_depth': 50, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.8620439770878263.
[I 2024-02-28 17:02:32,214] Trial 2 finished with value: 0.8628009685796577 and parameters: {'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.8628009685796577.
[I 2024-02-28 17:03:00,330] Trial 3 finished with value: 0.8623276083687005 and parameters: {'n_estimators': 400, 'max_depth': 70, 'min_samples_split': 8, 'min_samples_leaf': 2}. Best is trial 2 with value

In [ ]:
best_rf_params = study.best_params

In [ ]:
rf_tuned = RandomForestClassifier(**best_rf_params)

rf_tuned.fit(X_train, y_train)
y_pred = rf_tuned.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
auc

TypeError: RandomForestClassifier.__init__() got an unexpected keyword argument 'learning_rate'

#XGBoost Classifier

In [ ]:
se = StandardScaler()
scaled_X_train = se.fit_transform(X_train)
scaled_X_test = se.transform(X_test)

NameError: name 'StandardScaler' is not defined

In [ ]:
xgb_params = {'n_estimators': 200,
        'learning_rate': 0.01,
        'max_depth': 5,
        'subsample': 0.3,
        'colsample_bytree': 0.3,
        'gamma': 1e-6}

In [ ]:
xgb_model = XGBClassifier(random_state=42, **xgb_params, enable_categorical=True)

xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=1e-06, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred_xgb = xgb_model.predict_proba(X_test)[:, 1]
auc_xgb = roc_auc_score(y_test, y_pred_xgb)
auc_xgb

0.8589296132608591

##tuning

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'random_state': 10
    }

    xgb_model = XGBClassifier(**params, enable_categorical=True)

    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)

    return auc

In [ ]:
study = optuna.create_study(direction='maximize')

[I 2024-03-01 06:37:15,092] A new study created in memory with name: no-name-fccfe82e-68ad-403f-904e-f300386dd8c3


In [ ]:
study.optimize(objective, n_trials=30)

[I 2024-03-01 06:37:21,883] Trial 0 finished with value: 0.8493350251150302 and parameters: {'n_estimators': 207, 'learning_rate': 0.01661169306619972, 'max_depth': 3, 'subsample': 0.8695036559818463, 'colsample_bytree': 0.5041731701944661, 'gamma': 0.08718333738279883}. Best is trial 0 with value: 0.8493350251150302.
[I 2024-03-01 06:37:24,015] Trial 1 finished with value: 0.8502270974475494 and parameters: {'n_estimators': 236, 'learning_rate': 0.01581791709722112, 'max_depth': 3, 'subsample': 0.7820267593491408, 'colsample_bytree': 0.9699904377772403, 'gamma': 9.148493251241553e-05}. Best is trial 1 with value: 0.8502270974475494.
[I 2024-03-01 06:37:37,447] Trial 2 finished with value: 0.8625535127945801 and parameters: {'n_estimators': 205, 'learning_rate': 0.004105892557959294, 'max_depth': 9, 'subsample': 0.7367445791289681, 'colsample_bytree': 0.8531975274362495, 'gamma': 3.4867146955224075e-05}. Best is trial 2 with value: 0.8625535127945801.
[I 2024-03-01 06:37:45,191] Trial 

KeyboardInterrupt: 

In [ ]:
best_params = study.best_params

In [ ]:
best_params

{'n_estimators': 217,
 'learning_rate': 0.041509965338468416,
 'max_depth': 9,
 'subsample': 0.9927497897645347,
 'colsample_bytree': 0.7202038200230569,
 'gamma': 0.14508007721449825}

In [ ]:
parameters_xgb ={'n_estimators': 232,
 'learning_rate': 0.09330039417706945,
 'max_depth': 4,
 'subsample': 0.8180910137459942,
 'colsample_bytree': 0.7309531744186701,
 'gamma': 3.7710013857451275e-05}



In [ ]:
xgb_model_tuned = XGBClassifier(**parameters_xgb, random_state=10)

xgb_model_tuned.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7309531744186701, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None,
              gamma=3.7710013857451275e-05, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.09330039417706945, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=232, n_jobs=None,
              num_parallel_tree=None, random_state=10, ...)

In [ ]:
y_pred_xgb_tuned = xgb_model_tuned.predict_proba(X_test)[:, 1]
auc_xgb_tuned = roc_auc_score(y_test, y_pred_xgb_tuned)
auc_xgb_tuned

0.8754414180178599

#Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
gb_clf = GradientBoostingClassifier()

In [ ]:
gb_clf.fit(X_train, y_train)
y_pred_gb = gb_clf.predict_proba(X_test)[:, 1]
auc_gb = roc_auc_score(y_test, y_pred_gb)
auc_gb

0.86222485456681

##tuning

In [ ]:
from sklearn.model_selection import cross_validate

In [ ]:
def objective(trial):
  params = {
    "n_estimators": trial.suggest_int("n_estimators", 100, 500, step = 100),
    "learning_rate": trial.suggest_float("learning_rate", 1e-4, 0.3, log = True),
    "max_depth": trial.suggest_int("max_depth", 3, 9),
    "subsample": trial.suggest_float("subsample", 0.5, 0.9, step = 0.1),
    "random_state": 42,
    }
  # Perform cross validation
  gb_class = GradientBoostingClassifier(**params)

  gb_class.fit(X_train, y_train)
  y_pred = gb_class.predict_proba(X_test)[:, 1]
  auc = roc_auc_score(y_test, y_pred)

  return auc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2024-02-28 10:53:12,179] A new study created in memory with name: no-name-5ecf6964-d413-42b6-9a22-61ad1db3327e
[W 2024-02-28 11:04:42,589] Trial 0 failed with parameters: {'n_estimators': 3700, 'learning_rate': 0.009135368085344906, 'max_depth': 9, 'subsample': 0.9} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "<ipython-input-108-53e643c5f313>", line 12, in objective
    gb_class.fit(X_train, y_train)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 538, in fit
    n_stages = self._fit_stages(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 615, in _fit_stages
    raw_predictions = self._fit_stage(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_gb.py", line 257, in _fit_stage
    tree.fit(X, residual, sample_weight=sample_w

KeyboardInterrupt: 

In [ ]:
best_gb_params = study.best_params

In [ ]:
gb_tuned = GradientBoostingClassifier(**best_gb_params)

gb_tuned.fit(X_train, y_train)
y_pred = gb_tuned.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
auc

#Catboost Classifier

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model_catboost = CatBoostClassifier()
model_catboost.fit(X_train, y_train)

y_pred_catboost = model_catboost.predict_proba(X_test)[:, 1]
auc_catboost = roc_auc_score(y_test, y_pred_catboost)
auc_catboost


Learning rate set to 0.030697
0:	learn: 0.6838248	total: 43.8ms	remaining: 43.7s
1:	learn: 0.6744519	total: 70.4ms	remaining: 35.1s
2:	learn: 0.6649391	total: 96.9ms	remaining: 32.2s
3:	learn: 0.6568457	total: 122ms	remaining: 30.5s
4:	learn: 0.6492528	total: 151ms	remaining: 30s
5:	learn: 0.6411575	total: 175ms	remaining: 29s
6:	learn: 0.6345487	total: 204ms	remaining: 28.9s
7:	learn: 0.6282876	total: 228ms	remaining: 28.2s
8:	learn: 0.6225933	total: 253ms	remaining: 27.8s
9:	learn: 0.6171454	total: 277ms	remaining: 27.5s
10:	learn: 0.6116584	total: 315ms	remaining: 28.3s
11:	learn: 0.6054502	total: 343ms	remaining: 28.2s
12:	learn: 0.5993917	total: 367ms	remaining: 27.9s
13:	learn: 0.5952382	total: 392ms	remaining: 27.6s
14:	learn: 0.5909584	total: 421ms	remaining: 27.6s
15:	learn: 0.5869634	total: 447ms	remaining: 27.5s
16:	learn: 0.5833210	total: 472ms	remaining: 27.3s
17:	learn: 0.5785082	total: 497ms	remaining: 27.1s
18:	learn: 0.5749503	total: 523ms	remaining: 27s
19:	learn: 0.5

0.8760481272638548

In [ ]:
EARLY_STOPPING_ROUND = 100

In [ ]:
def objective(trial):
    params = {
        "iterations": 1000,
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.1, log=True),
        "depth": trial.suggest_int("depth", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.05, 1.0),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
    }

    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)

    return auc

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, n_jobs=-1)

In [ ]:
parameters_catboost = {'learning_rate': 0.04, 'depth': 6, 'subsample': 0.6, 'colsample_bylevel': 0.5, 'min_data_in_leaf': 80}


In [ ]:
model_catboost_tuned = CatBoostClassifier(**parameters_catboost)
model_catboost_tuned.fit(X_train, y_train)

y_pred_catboost_tuned = model_catboost_tuned.predict_proba(X_test)[:, 1]
auc_catboost_tuned = roc_auc_score(y_test, y_pred_catboost_tuned)
auc_catboost_tuned

#SVM Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [ ]:
se = StandardScaler()
scaled_X_train = se.fit_transform(X_train)
scaled_X_test = se.transform(X_test)

parameters_catboost

In [ ]:
svm = SVC(probability=True)

svm.fit(scaled_X_train, y_train)
y_pred = svm.predict_proba(scaled_X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred)
auc



0.8671135494934661

In [ ]:
y_pred = svm.predict_proba(scaled_X_val)[:, 1]
auc = roc_auc_score(y_val, y_pred)
auc

0.7945729289397336

#MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier


In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(15),
	max_iter=3000)

In [ ]:
mlp.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=15, max_iter=3000)

In [ ]:
y_pred_mlp = mlp.predict_proba(X_test)[:, 1]
auc_mlp = roc_auc_score(y_test, y_pred_mlp)
auc_mlp

0.5001249932242193

#Voting Classifier

In [ ]:
from sklearn.ensemble import VotingClassifier

In [ ]:
estimators = [('XGBClassifier', xgb_model_tuned), ('CatBoostClassifier', model_catboost_tuned)]

In [ ]:
VC_soft = VotingClassifier(estimators = estimators, voting ='soft')
VC_soft.fit(X_train, y_train)
# y_pred_soft = VC_soft.predict(X_test)
# auc_soft = roc_auc_score(y_test, y_pred_soft)
# auc_soft

#Stacking Classifier

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier

In [ ]:
estimators = [('Random Forest', RandomForestClassifier(**parameters_rf)),
              ('XGBoost', XGBClassifier(**parameters_xgb)),
              ('CatBoost', CatBoostClassifier(**parameters_catboost)),
              ('Support Vector', SVC(probability=True))]

In [ ]:
XGB = XGBClassifier(**parameters_xgb)

In [ ]:
SC = StackingClassifier(estimators=estimators, final_estimator=XGB,cv=6)
SC.fit(scaled_X_train, y_train)
y_pred = SC.predict(scaled_X_test)
auc_stack = roc_auc_score(y_test, y_pred)
auc_stack

Выходные данные были обрезаны до нескольких последних строк (5000).
0:	learn: 0.6808990	total: 22.1ms	remaining: 22.1s
1:	learn: 0.6697780	total: 43.3ms	remaining: 21.6s
2:	learn: 0.6579013	total: 64.6ms	remaining: 21.5s
3:	learn: 0.6480993	total: 84.4ms	remaining: 21s
4:	learn: 0.6378790	total: 107ms	remaining: 21.3s
5:	learn: 0.6295992	total: 127ms	remaining: 21s
6:	learn: 0.6212975	total: 150ms	remaining: 21.2s
7:	learn: 0.6140941	total: 168ms	remaining: 20.9s
8:	learn: 0.6072154	total: 194ms	remaining: 21.3s
9:	learn: 0.6013918	total: 213ms	remaining: 21.1s
10:	learn: 0.5956237	total: 240ms	remaining: 21.5s
11:	learn: 0.5898155	total: 260ms	remaining: 21.4s
12:	learn: 0.5839134	total: 282ms	remaining: 21.4s
13:	learn: 0.5781732	total: 302ms	remaining: 21.3s
14:	learn: 0.5706049	total: 324ms	remaining: 21.3s
15:	learn: 0.5645611	total: 343ms	remaining: 21.1s
16:	learn: 0.5599752	total: 367ms	remaining: 21.2s
17:	learn: 0.5544013	total: 387ms	remaining: 21.1s
18:	learn: 0.5503641	tot

0.8474494736080517

#autogluon

In [ ]:
!pip install autogluon


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.1/229.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.0/306.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.7/416.7 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.5/120.5 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor


In [ ]:
train,test= train_test_split(train_data, test_size=0.2, random_state=42, shuffle=True, stratify=train_data['DiagPeriodL90D'])


In [ ]:
label = 'DiagPeriodL90D'


In [ ]:
predictor = TabularPredictor(label=label,eval_metric ='roc_auc').fit(train,presets='best_quality')
results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20240301_072026"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 3600 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: AutogluonModels/ag-20240301_072026/ds_sub_fit/sub_fit_ho.
Beginning AutoGluon training ... Time limit = 900s
AutoGluon will save models to "AutogluonModels/ag-20240301_072026/ds_sub_fit/sub_fit_ho"
=================== System Info ===================
AutoGluon Version:  1.

*** Summary of fit() ***
Estimated performance of each model:
                          model  score_val eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0           WeightedEnsemble_L2   0.811697     roc_auc       7.910519  1421.165047                0.003059           5.273566            2       True         22
1           LightGBM_r96_BAG_L1   0.805760     roc_auc       0.635419    51.909910                0.635419          51.909910            1       True         19
2               CatBoost_BAG_L1   0.801967     roc_auc       0.185964   129.021427                0.185964         129.021427            1       True          7
3          CatBoost_r177_BAG_L1   0.801923     roc_auc       0.148374   120.020098                0.148374         120.020098            1       True         14
4         NeuralNetTorch_BAG_L1   0.801447     roc_auc       0.708972   171.147193                0.708972         171.147193        

In [ ]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L2,0.811697,roc_auc,7.910519,1421.165047,0.003059,5.273566,2,True,22
1,LightGBM_r96_BAG_L1,0.805760,roc_auc,0.635419,51.909910,0.635419,51.909910,1,True,19
2,CatBoost_BAG_L1,0.801967,roc_auc,0.185964,129.021427,0.185964,129.021427,1,True,7
3,CatBoost_r177_BAG_L1,0.801923,roc_auc,0.148374,120.020098,0.148374,120.020098,1,True,14
4,NeuralNetTorch_BAG_L1,0.801447,roc_auc,0.708972,171.147193,0.708972,171.147193,1,True,12
5,LightGBMXT_BAG_L1,0.801345,roc_auc,0.171019,48.992050,0.171019,48.992050,1,True,3
6,XGBoost_BAG_L1,0.799970,roc_auc,0.342870,80.327472,0.342870,80.327472,1,True,11
7,NeuralNetTorch_r22_BAG_L1,0.799596,roc_auc,0.914122,254.541652,0.914122,254.541652,1,True,20
8,LightGBM_r131_BAG_L1,0.799005,roc_auc,0.491816,92.123951,0.491816,92.123951,1,True,16
9,LightGBM_BAG_L1,0.798064,roc_auc,0.148318,58.251971,0.148318,58.251971,1,True,4


NameError: name 'WeightedEnsemble_L2' is not defined

In [ ]:
df = predictor.predict_proba(test_data)
df.shape

,0,1
0,0.201597,0.798403
1,0.264608,0.735392
2,0.269376,0.730624
3,0.268922,0.731078
4,0.199024,0.800976


In [ ]:
df.shape

(5792, 2)

In [ ]:
test_data[]

InvalidIndexError: (slice(None, None, None), 1)

#Submission

In [ ]:
test_data['patient_id']

0       573710
1       593679
2       184532
3       447383
4       687972
5       281312
6       492714
7       378266
8       291550
9       612272
10      240105
11      277939
12      504153
13      287269
14      108727
15      598629
16      805201
17      565624
18      689369
19      252028
20      830503
21      777454
22      931410
23      894910
24      257477
25      373935
26      929645
27      164064
28      558677
29      707003
30      289528
31      340932
32      182933
33      887761
34      249345
35      750357
36      162816
37      346740
38      496264
39      900330
40      630418
41      582166
42      397291
43      617035
44      903270
45      628977
46      664844
47      571761
48      284488
49      573502
50      914179
51      533295
52      673470
53      394209
54      165379
55      335343
56      521842
57      513091
58      144195
59      114345
60      611289
61      430995
62      755430
63      170191
64      739107
65      760626
66      99

In [ ]:
test_data['DiagPeriodL90D'] = df[1]
submission = test_data[['patient_id','DiagPeriodL90D']]
submission.to_csv('/content/drive/MyDrive/WinDS/submission_ana_27.csv', index=False)


In [ ]:
test_data = se.transform(test_data)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
sub_proba = xgb_model_tuned.predict_proba(test_data.drop(columns=['patient_id']))
test_data['DiagPeriodL90D'] = sub_proba[:, 1]
submission = test_data[['patient_id','DiagPeriodL90D']]
submission.head()

,patient_id,DiagPeriodL90D
0,573710,0.801780
1,593679,0.633238
2,184532,0.706015
3,447383,0.576189
4,687972,0.824491


In [ ]:
submission.shape

(5792, 2)

In [ ]:
submission.to_csv('/content/drive/MyDrive/WinDS/submission_ana_27.csv', index=False)